# August 10, 2022: generate trial level responses of MAX data for ABA rois

In [1]:
import os 
import sys
from os.path import join as pjoin
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle, random

from nltools import mask
from nilearn import image, masking

# plotting
import matplotlib.pyplot as plt
plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr #CITE ITS PAPER IN YOUR MANUSCRIPT

# main dirs
proj_dir = pjoin(os.environ['HOME'], 'network_dynamics')
results_dir = f"{proj_dir}/results"
month_dir = f"{proj_dir}/nb/aug22"

# folders
sys.path.insert(0, proj_dir)
import helpers.dataset_utils as dataset_utils

/home/govindas/venvs/expln-ai3.9/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


## MAX rois

In [2]:
max_roi_name_file = (
    f"{os.environ['HOME']}/parcellations/MAX_85_ROI_masks/ROI_names.txt"
)
max_roi_names = pd.read_csv(max_roi_name_file, names=['roi_name']).values.squeeze()
max_roi_names = list(max_roi_names)
max_roi_names

['R med. OFC',
 'L med. OFC',
 'R lat. OFC',
 'L lat. OFC',
 'M vmPFC1',
 'M vmPFC2',
 'R dlPFC',
 'L dlPFC',
 'R FEF',
 'L FEF',
 'R pregenual ACC',
 'L pregenual ACC',
 'R ACC',
 'L ACC',
 'R ant. MCC',
 'L ant. MCC',
 'R post. MCC',
 'L post. MCC',
 'R pre-SMA',
 'L pre-SMA',
 'M PCC',
 'R PCC/precuneus',
 'L PCC/precuneus',
 'R PCC',
 'L PCC',
 'R Ventral striatum',
 'L Ventral striatum',
 'R ant. Putamen',
 'L ant. Putamen',
 'R post. Putamen',
 'L post. Putamen',
 'R ant. Caudate',
 'L ant. Caudate',
 'R post. Caudate',
 'L post. Caudate',
 'R IFG-1',
 'L IFG-1',
 'R IFG-2',
 'L IFG-2',
 'R IFG-3',
 'L IFG-3',
 'R IFG-4',
 'L IFG-4',
 'R IFG-5',
 'L IFG-5',
 'R IFG-6',
 'L IFG-6',
 'R ant. ventral Insula',
 'L ant. ventral Insula',
 'R ant. dorsal Insula',
 'L ant. dorsal Insula',
 'R mid/post Insula',
 'L mid/post Insula',
 'R BST',
 'L BST',
 'R CeMe Amygdala',
 'L CeMe Amygdala',
 'R BLBM Amygdala',
 'L BLBM Amygdala',
 'R ant. Hippocampus',
 'L ant. Hippocampus',
 'R Hippocam

In [3]:
class ARGS(): pass
args = ARGS()

with open(f"{proj_dir}/data/max/exploratory_data_roi_indices.pkl", 'rb') as f:
    args.up_roi_idxs, args.down_roi_idxs, args.roi_idxs = pickle.load(f)
    args.num_rois = len(args.roi_idxs)

## ABA rois

In [4]:
set_name = 'aba'
aba_roi_name_file = (
    f"/home/govindas/vscode-BSWIFT-mnt/ABA/ROI_mask/ABA_36ROIs_gm.txt"
)
aba_roi_names =  pd.read_csv(
    aba_roi_name_file,
    delimiter='\t'
)[['sHemi', 'ROI']]
aba_roi_names = (aba_roi_names['sHemi'] + ' ' + aba_roi_names['ROI']).values
aba_roi_names = list(aba_roi_names)
aba_roi_names

['R Hypothalamus',
 'L Hypothalamus',
 'R ACC',
 'L ACC',
 'R ant. MCC',
 'L ant. MCC',
 'R post. MCC',
 'L post. MCC',
 'R PCC',
 'L PCC',
 'R Ventral striatum',
 'L Ventral striatum',
 'R IFG-5',
 'L IFG-5',
 'R ant. ventral Insula',
 'L ant. ventral Insula',
 'R ant. dorsal Insula',
 'L ant. dorsal Insula',
 'R mid/post Insula',
 'L mid/post Insula',
 'R BST',
 'L BST',
 'R CeMe Amygdala',
 'L CeMe Amygdala',
 'R BLBM Amygdala',
 'L BLBM Amygdala',
 'R ant. Hippocampus',
 'L ant. Hippocampus',
 'R PAG',
 'L PAG',
 'R Pulvinar',
 'L Pulvinar',
 'R VTA-SNc',
 'L VTA-SNc',
 'Ant. vmPFC',
 'Post. vmPFC']

34 out of 36 ABA rois are taken from the MAX rois, and two rois are added: Pulvinar and VTA-SNc.

## ABA to MAX

In [5]:
'''
map aba onto max
'''
# for 33 rois
aba_to_max_idxs = {
    aba_idx:max_roi_names.index(aba_roi) 
    for aba_idx, aba_roi in enumerate(aba_roi_names) 
    if aba_roi in max_roi_names
}
# for the two additional rois
for aba_roi_idx in range(30, 34):
    aba_to_max_idxs[aba_roi_idx] = None
# for vmPFC
aba_to_max_idxs[34] = 4
aba_to_max_idxs[35] = 5

display(aba_to_max_idxs)

'''
map max to aba
'''
max_to_aba_idxs = {
    v:k for k, v in aba_to_max_idxs.items()
}
display(max_to_aba_idxs)

{0: 65,
 1: 66,
 2: 12,
 3: 13,
 4: 14,
 5: 15,
 6: 16,
 7: 17,
 8: 23,
 9: 24,
 10: 25,
 11: 26,
 12: 43,
 13: 44,
 14: 47,
 15: 48,
 16: 49,
 17: 50,
 18: 51,
 19: 52,
 20: 53,
 21: 54,
 22: 55,
 23: 56,
 24: 57,
 25: 58,
 26: 59,
 27: 60,
 28: 79,
 29: 80,
 30: None,
 31: None,
 32: None,
 33: None,
 34: 4,
 35: 5}

{65: 0,
 66: 1,
 12: 2,
 13: 3,
 14: 4,
 15: 5,
 16: 6,
 17: 7,
 23: 8,
 24: 9,
 25: 10,
 26: 11,
 43: 12,
 44: 13,
 47: 14,
 48: 15,
 49: 16,
 50: 17,
 51: 18,
 52: 19,
 53: 20,
 54: 21,
 55: 22,
 56: 23,
 57: 24,
 58: 25,
 59: 26,
 60: 27,
 79: 28,
 80: 29,
 None: 33,
 4: 34,
 5: 35}

In [6]:
for max_roi in args.up_roi_idxs:
    if max_roi not in max_to_aba_idxs.keys(): continue
    print(max_to_aba_idxs[max_roi])

2
3
4
5
6
7
14
15
16
17
20
21
28
29


In [7]:
'''
group aba rois separately from max and save the grouping
'''
args.up_roi_idxs = np.array(
    [2, 3] + # ACC
    list(range(4, 8)) + # MCC
    [12, 13] + # IFG-5
    list(range(14, 20)) + # insula
    [20, 21] + # BST
    [30, 31] # Pulvinar
)

args.down_roi_idxs = np.array(
    [8, 9] + # PCC
    [26, 27] + # ant. Hippocampus
    [34, 35] # vmPFC
)

args.zero_roi_idxs = np.array(
    [0, 1] + # Hypothalamus
    [10, 11] + # ventral striatum
    list(range(22, 26)) + # amygdala
    [28, 29] + # PAG
    [32, 33] # VTA-SNc
)

args.roi_idxs = np.concatenate([args.up_roi_idxs, args.zero_roi_idxs, args.down_roi_idxs])

In [9]:
len(args.roi_idxs)

36

In [8]:
with open(f"{proj_dir}/data/max/exploratory_data_aba_roi_indices.pkl", 'wb') as f:
    pickle.dump([args.up_roi_idxs, args.down_roi_idxs, args.zero_roi_idxs, args.roi_idxs], f)